<a href="https://colab.research.google.com/github/cosmina98/PhD/blob/main/MLP_on_graph_classification_using_PyTorch/Wrapped_up_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raw Pytorch

In [245]:
!pip install pytorch-lightning --quiet


     |████████████████████████████████| 585 kB 15.1 MB/s 
     |████████████████████████████████| 419 kB 49.6 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 140 kB 68.1 MB/s 
     |████████████████████████████████| 1.1 MB 52.0 MB/s 
     |████████████████████████████████| 144 kB 53.0 MB/s 
     |████████████████████████████████| 271 kB 53.9 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 


In [246]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time


In [267]:
#definining the MLP class
class MLP(nn.Module):
  '''
    Multilayer Perceptron.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Flatten(),
      nn.Linear(32 * 32 * 3, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 10)
    )


  def forward(self, x):
    '''Forward pass'''
    return self.layers(x)

In [283]:
#download and normalise data
torch.manual_seed(42)
ROOT = '.data'


# Prepare CIFAR-10 dataset
trainset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=False, num_workers=2)

testset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [274]:

# Initialize the MLP
mlp = MLP()
  
# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr=1e-3,momentum=0.9 , nesterov=True, weight_decay=0)
optimizer_2 = torch.optim.Adam(mlp.parameters(), lr=1e-4)


In [381]:
def train(optimizer):
    for epoch in range(2):  # loop over the dataset 2 times

       running_loss = 0.0
       for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + compute loss+ backward + optimize
        outputs = mlp(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

    print('Finished Training')

def test(model):
    classes = ('plane', 'car', 'bird', 'cat',
              'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            estimation, predictions = torch.max(outputs, 1)
            print(np.log(estimation))
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [380]:
train(optimizer)
test(mlp)

[1,  2000] loss: 1.459
[1,  4000] loss: 1.413
[1,  6000] loss: 1.427
[2,  2000] loss: 1.391
[2,  4000] loss: 1.352
[2,  6000] loss: 1.373
Finished Training
Accuracy for class: plane is 50.5 %
Accuracy for class: car   is 66.5 %
Accuracy for class: bird  is 32.4 %
Accuracy for class: cat   is 49.2 %
Accuracy for class: deer  is 44.1 %
Accuracy for class: dog   is 30.5 %
Accuracy for class: frog  is 61.7 %
Accuracy for class: horse is 71.3 %
Accuracy for class: ship  is 71.2 %
Accuracy for class: truck is 60.3 %


# Wrapped up Pytorch

In [2]:
# Prepare CIFAR-10 dataset
trainset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=False, num_workers=2)

testset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/cifar-10-python.tar.gz to /content
Files already downloaded and verified


In [51]:
class Trainer():
    def __init__ (self):
       
      self.batch_size=2000
      self.epochs=2
      #used for label count 
      self.num_classes=0
      self.verbose=True
        
      self.loss_function = nn.CrossEntropyLoss()
       
      #common  to most optimsers
      self.learning_rate=1e-3
      self.weight_decay=0
      self.running_loss = 0.0
    
      #for sgd  type of optimisers
     
      #error when trying to set the momentum using self.momentum
      self.nesterov= True,
      self.dampening=0
     
      #for adam optimiser
      self.betas=(0.9,0.99)
      self.eps=1e-8
      self.amsgrad=False

      self.mlp=self.MLP()
      self.optimiser=None

    class MLP(nn.Module):
      '''
        Multilayer Perceptron.
      '''
      def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(32 * 32 * 3, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 10)
        )


      def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

    def print_self(self):
        print(mlp)
          
    def set_optimiser(self,optimiser_choice):
        #playing around with adam and sgd only at the moment
        #there is only two of them 
        
        if optimiser_choice.lower().strip()=='adam':
          self.optimiser=torch.optim.Adam(self.mlp.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad)
        elif optimiser_choice.lower().strip()=='sgd':
          self.optimiser=torch.optim.SGD(self.mlp.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=0.9,nesterov=self.nesterov , dampening=self.dampening)
          return self.optimiser



    def get_params(self): #get parameters
       if (self.optimiser != None) :
         return  (self.props(),self.optimiser.state_dict(), self.mlp.state_dict)
       else: return self.mlp.state_dict() 
  
    def fit(self,trainloader):
        for epoch in range(self.epochs):  # loop over the dataset 2 times

          running_loss =self.running_loss
          for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
             
            # zero the parameter gradients
            self.optimiser.zero_grad()

            # forward + compute loss+ backward + optimize
            outputs = self.mlp(inputs)
            loss = self.loss_function(outputs, labels)
            loss.backward()
            self.optimiser.step()
            # print statistics
            running_loss += loss.item()
            if i % self.batch_size == 1999:    # print every 2000 mini-batches
                if self.verbose==True:
                  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / self.batch_size:.3f}')
                running_loss = 0.0


    def predict(self,testloader): #Predict using the multi-layer perceptron classifier.
      prediction_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = self.mlp(images)
              estimation, predictions = torch.max(outputs, 1)
              prediction_list.append(predictions)
      return prediction_list #torch.stack(prediction_list)


    def predict_log_proba(self,loader):  #	Return the log of probability estimates.
        y_prob=self.predict_proba(loader)
        log_proba=np.log(y_prob)
        return log_proba
 

    def predict_proba(self,loader):	#Probability estimates.
      probabilities_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in loader:
              images, labels = data
              outputs = self.mlp(images)
              estimation, predictions = torch.max(outputs, 1)
              probabilities_list.append(estimation)
      return probabilities_list
      

    def score(self,loader): #Return the mean accuracy on the given test data and labels.
        prediction_list=[]
        if self.num_classes==0:
            classes = ('plane', 'car', 'bird', 'cat',
                    'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
            # prepare to count predictions for each class
            correct_pred = {classname: 0 for classname in classes}
            total_pred = {classname: 0 for classname in classes}
        #use the class number as key
        else: 
            classes=tuple(np.arange(self.num_classes))  
            correct_pred = {classname: 0 for classname in np.arange(num_classes)}
            total_pred= {classname: 0 for classname in np.arange(num_classes)}

        # again no gradients needed
        with torch.no_grad():
            for data in loader:
                images, labels = data
                outputs = self.mlp(images)
                estimation, predictions = torch.max(outputs, 1)
                prediction_list.append(predictions)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[classes[label]] += 1
                    total_pred[classes[label]] += 1
        accuracies={}
        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            accuracies[classname]=accuracy
            if self.verbose:
              print(f'Accuracy for class: {classname} is {accuracy:.1f} %')
        

    def set_params(self, attr, value): #sets parameters
        setattr(self, attr, value)
        
    def props(cls):   
        return [i for i in cls.__dict__.items() if i[:1] != '_']




In [ ]:
model=Trainer()
model.set_optimiser('adam')
model.set_params('verbose', False)
model.get_params()
model.fit(trainloader)
model.predict(testloader)
model.score(testloader)
model.predict_log_proba(testloader)
model.predict_log_proba(testloader)